In [1]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import BertForMaskedLM, BertTokenizer

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
with open("../../data/pseudowords/CoMaPP_all_bert.json") as json_file:
    data = json.load(json_file)
    
data = [{"example": d["target1"], "query": (" ".join(d["query"].split()[:d["query_idx"]]) + " " + d["label"] + " " + " ".join(d["query"].split()[d["query_idx"]+1:])).strip(), "pseudoword": d["label"]} for d in data]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,query,pseudoword
0,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
1,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
2,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
3,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
4,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
83378,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99
83379,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99
83380,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99
83381,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99


In [3]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,query,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99
99,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99
99,Rund 45 . 000 Dollar soll der Byton - SUV kost...,Rund 45 . 000 Dollar soll der Byton - SUV kost...,er99


In [4]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'query': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [Und schon gar nicht mit der Mehrwertsteuer .,...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                     query  
construction pseudoword                                                     
5            Und5        [Und5 schon gar nicht [MASK] der Mehrwertsteue...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [[MASK] hat Afghanistan nicht stabilisiert und...  
             nicht5      [[MASK] hat Afghanistan nicht5 stabilisiert un...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [[MASK] ist Silber , reden ist Gold1884 ., Sch...  
             Silber1884  [[MASK] ist Silber1884 , reden ist Gold ., Sch...  
             ist1884     [[MASK] ist1884 Silber , reden ist Gold ., Sch...  
1986         kaum1986    [[MASK] Vorhut vor 20.000 Jahren war für das R...  
1987         sehr1987    [[MASK] Menschen können sich sehr1987 wohlfühl...  

[581 rows x 2 columns]

In [5]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ..."
...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z..."


In [6]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)  # TODO: Aussortierte Definitions betrachten!
examples

example  \
construction pseudoword                                                      
5            Und5        [Und schon gar nicht mit der Mehrwertsteuer .,...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                     query  \
construction pseudoword                                                      
5            Und5        [Und5 schon gar nicht [MASK] der Mehrwertsteue...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [[MASK] hat Afghanistan nicht stabilisiert und...   
             nicht5      [[MASK] hat Afghanistan nicht5 stabilisiert un...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [[MASK] ist Silber , reden ist Gold1884 ., Sch...   
             Silber1884  [[MASK] ist Silber1884 , reden ist Gold ., Sch...   
             ist1884     [[MASK] ist1884 Silber , reden ist Gold ., Sch...   
1986         kaum1986    [[MASK] Vorhut vor 20.000 Jahren war für das R...   
1987         sehr1987    [[MASK] Menschen können sich sehr1987 wohlfühl...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[581 rows x 3 columns]

##### Generieren neuer Sätze:

In [7]:
pseudowords = []
for i in range(15):
    pseudowords.append(np.load(f"../../data/pseudowords/bert/pseudowords_comapp_bert_{i*37}_{i*37+37}.npy"))
pseudowords = np.concatenate(pseudowords)
pseudowords

array([[-2.0389168 , -1.6559315 ,  2.3402972 , ...,  1.9295877 ,
        -0.2874232 ,  0.1127736 ],
       [ 2.2873163 ,  1.1256251 , -0.72309786, ..., -0.9724281 ,
        -0.6235189 ,  0.5858617 ],
       [-0.72157246,  1.1234709 , -0.19191697, ..., -0.44904602,
        -0.63462436, -0.51367843],
       ...,
       [ 1.5758858 ,  3.0587277 ,  1.1784296 , ..., -3.5326967 ,
         2.022733  ,  0.7190625 ],
       [-0.02080958,  2.2421186 ,  0.12035339, ..., -0.83508754,
        -0.4160101 ,  0.07736167],
       [ 1.2608864 ,  3.31698   , -3.9412627 , ..., -0.71156853,
        -2.0257826 ,  2.4922867 ]], dtype=float32)

In [8]:
csv_data = []
for i in range(1, 16):
    csv_data.append(pd.read_csv(f"../../data/pseudowords/bert/order_bert_{i}.csv", sep=";", index_col=0, header=None, quotechar="|", names=["order", "label"]))
csv_data = pd.concat(csv_data)
csv_data

,label
order,
0,"""""Was13"
1,"""647"
2,"""Wir-äh-spielen-äh-in-der-äh-Champions-League647"
3,(1597
4,(1600
...,...
550,wohl1134
551,wollen1029
552,wäre130


In [9]:
bert_tokens = [d[0] for d in csv_data.values]

bert_tokens, len(bert_tokens)

(['""Was13',
  '"647',
  '"Wir-äh-spielen-äh-in-der-äh-Champions-League647',
  '(1597',
  '(1600',
  '(1602',
  '(1624',
  '(1637',
  '(1639',
  '(1641',
  '(1643',
  '(1645',
  '(379',
  '(579',
  '(581',
  '(584',
  '(590',
  '(592',
  '(600',
  '(886',
  '(889',
  '(892',
  '(900',
  '(905',
  '(907',
  '(909',
  '(911',
  '(917',
  '(919',
  '(921',
  '(923',
  ')1597',
  ')1600',
  ')1602',
  ')1624',
  ')1637',
  ')1639',
  ')1641',
  ')1643',
  ')1645',
  ')1792',
  ')379',
  ')579',
  ')581',
  ')584',
  ')590',
  ')592',
  ')600',
  ')886',
  ')889',
  ')892',
  ')900',
  ')905',
  ')907',
  ')909',
  ')911',
  ')917',
  ')919',
  ')921',
  ')923',
  ')«579',
  ',1459',
  ',973',
  '-128',
  '-651',
  '-654',
  '-875',
  '-973',
  ':595',
  ':875',
  ':973',
  'Abstand683',
  'Allein20',
  'Aller1630',
  'Als1315',
  'Als133',
  'Als1770',
  'Am488',
  'Am492',
  'Am500',
  'Amerika605',
  'Anstatt320',
  'Art129',
  'Arzt1509',
  'Augenblick1301',
  'Ausmaß1777',
  'BRUTAL150

Load the vanilla bert-german model:

In [10]:
model = BertForMaskedLM.from_pretrained('bert-base-german-cased', return_dict=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
model.bert.embeddings.word_embeddings

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(30000, 768, padding_idx=0)

Add to existing embeddings:

In [11]:
combined_embeddings = torch.cat((model.bert.embeddings.word_embeddings.weight, torch.tensor(pseudowords)), dim=0)
model.bert.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(combined_embeddings)
model.bert.embeddings.word_embeddings

Embedding(30555, 768)

Add to existing tokens:

In [12]:
tokenizer.add_tokens(bert_tokens)
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 30555. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(30555, 768)

Complete the masks:

In [13]:
def complete_masks(row):
    try:
        output_texts = []
        scores = []
        for query, example in list(zip(row["query"], row["example"])):
            tokenized_query = ["[CLS]"] + tokenizer.tokenize(query) + ["[SEP]"]
            masked_index = tokenized_query.index("[MASK]")
            input_ids = tokenizer.convert_tokens_to_ids(tokenized_query)
            input_ids = torch.tensor([input_ids])
            
            # Predict the most probable word that is not part of the new embeddings:
            with torch.no_grad():
                outputs = model(input_ids)
                predictions = outputs.logits
            predicted_token_probs = predictions[0, masked_index]
            vocab_size = len(tokenizer)
            wanted_vocab_size = vocab_size - len(tokenizer.get_added_vocab())  # 27000 - 30000: unused tokens; 30000+: new tokens
            
            # Find the top 5 predicted tokens with IDs lower than 28997
            found = 0
            for i in range(vocab_size):
                if found:#  >= 5:
                    break
                token_id = torch.argsort(predicted_token_probs, descending=True)[i].item()
                if token_id < wanted_vocab_size:
                    predicted_token = tokenizer.convert_ids_to_tokens([token_id])[0]
                    if "unused_" in predicted_token:  # unused_token, unused_punctuation
                        continue
                    found += 1
                    output_text = tokenized_query[:masked_index] + [predicted_token] + tokenized_query[masked_index+1:]
                    score = predicted_token_probs[token_id].item()
                    print(row["pseudoword"], found, " ".join(output_text), score)
                    output_texts.append(output_text)
                    scores.append(score)
        
        return pd.Series({'construction': row['construction'], 'pseudoword': row['pseudoword'], 'example': row['example'], 'generated': output_texts, 'score': [float(score) for score in scores], 'definition': row['definition']})
    except Exception as e:  # TODO Entfernen, sobald alle Pseudowords da sind
        print(".", end="")
        return pd.Series({'construction': row['construction'], 'pseudoword': row['pseudoword'], 'example': row['example'], 'generated': [str(e)], 'score': [-1.0], 'definition': row['definition']})

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset.progress_apply(complete_masks, axis=1)  # TODO use "pseudoword" as index
pseudoword_output_scores

  0%|          | 0/581 [00:00<?, ?it/s]

Und5 1 [CLS] Und5 schon gar nicht wegen der Mehrwertsteuer . [SEP] 17.02823829650879
Und5 1 [CLS] Und5 schon gar nicht mit der Mehrwertsteuer . [SEP] 18.764745712280273
Und5 1 [CLS] Und5 schon gar nicht mit der Zeit . [SEP] 10.610733032226562
Und5 1 [CLS] Und5 schon gar nicht dadurch , dass man die Patienten dafür dann nicht anderweitig zur Kasse bittet . [SEP] 14.959037780761719
Und5 1 [CLS] Und5 schon gar nicht darauf als dass man die Patienten dafür dann nicht anderweitig zur Kasse bittet . [SEP] 10.12462329864502
Und5 1 [CLS] Und5 schon gar nicht darauf , dass man die Patienten dafür dann nicht anderweitig zur Kasse bittet . [SEP] 18.30353546142578
Und5 1 [CLS] Und5 schon gar nicht darauf , dass man die Patienten dafür dann nicht anderweitig zur Kasse bittet . [SEP] 16.84368133544922
Und5 1 [CLS] Und5 schon gar nicht darauf , dass man die Patienten dafür dann nicht anderweitig zur Kasse bittet . [SEP] 18.3360595703125
Und5 1 [CLS] Und5 schon gar nicht darauf , dass man die Kunden d

KeyboardInterrupt: 

In [ ]:
examples = pseudoword_output_scores[["pseudoword", "generated", "score"]]

examples

#### Anzahl neuer Sätze

In [ ]:
examples.to_csv(f"../../out/comapp/data_bert.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/data_bert.xlsx")

In [ ]:
pseudoword_output_scores.to_csv("../../out/comapp/data_bert_complete.tsv", sep="\t", decimal=",")